
# Submission instructions

In this programming assignment, you will create a simple search engine. More specifically, you will implement an index over a corpus of webpages, and also compress the index using variable length encoding. Both uncompressed and compressed indices will be tested on retrieval with Boolean conjunctive queries. The assignment is object-oriented and relies on python concepts such as functions, files and classes. Our TA, Makanjuola, has compiled a list of tutorials on these concepts:

1. A short video: [Learn python in 1 hour](https://www.youtube.com/watch?v=kqtD5dpn9C8)
2. [These docs](https://docs.python.org/3.7/reference/datamodel.html#special-method-names) describe special functions such as `__getitem__` , etc. 
3. This [short tutorial](https://www.omkarpathak.in/2018/04/11/python-getitem-and-setitem/) is also useful to get started with special functions.
4. Here is a very helpful tutorial on Python [file handling](https://www.programiz.com/python-programming/file-operation).
5. [This youtube video](https://www.youtube.com/watch?v=wfcWRAxRVBA) and [this article](https://www.w3schools.com/python/python_classes.asp) can help get you started with Classes in Python.
6. All tutorials (and more) are categorized in this [Google doc](https://docs.google.com/document/d/1Q_aw1bW2Dx3eUMnlQLYl1p8iMyf4lXkBDjNfk688jHw/edit?usp=sharing).

The HW is due **Wednesday, September 29, 2021 @ 11:59 pm**. You can form teams of two or work individually. Note that there exists no difference in terms of grading, i.e., we will grade the same for one and two person teams. Only one of the team members needs to submit the HW. 
 

Instructions on how to submit will be given soon. We will use an autograding system. Please do **NOT** change class and method names, otherwise autograding will fail. ONLY edit the parts with **"### Begin your code"** and **"### End your code"** comments. Note that all aspects of this assignment are covered by the [Honor Code](https://graduateschool.vt.edu/content/dam/graduateschool_vt_edu/graduate-honor-system/Constitution2021.pdf) and that the autograding system will also perform screening checks for code similarity. We will also inspect each student's code for implementation correctness. Finally, note that we will increase office hours before the HW deadline.

## Declare your teammate here 
Include the name and email for your team member. If you are working independenty, write "Myself"
> My teammate for this HW is: `Divyansh Mishra, divyanshmishra@vt.edu`

## Grading breakdown

**Autograder (60%)**: To ensure your indices are built correctly, both the uncompressed and compressed index versions of your implementations will be tested on 20 Boolean conjunctive queries of one or multiple terms. For those queries, you will get 1.5% of the final grade for each query you answer correctly, for a total of 60% of your grade (30% for the compressed index and 30% for the uncompressed index). 

**Code inspection (40%)**: We will also grade each function implementation individually. The grading breakdown is as follows:
1. `IdMap` - 4%
2. `BSBIIndex parse_block` - 4%
3. `InvertedIndexWriter append` - 2%
4. `BSBIIndex invert_write` - 4%
5. `InvertedIndexIterator` - 2%
6. `BSBIIndex merge` - 4%
7. `InvertedIndexMapper _get_postings_list` - 2%
8. `sorted_intersect` - 4%
9. `BSBIIndex retrieve` - 4%
10. `CompressedPostings` - 4%
11. Index size & Performance Questions (Q1-Q4) - 1.5% each question, for a total for 6% (4 x 1.5)


Finally note that in order to get full score:
- The `retrieve` function should NOT load the whole index into memory  but rather load the postings lists of just the query terms.
- The `merge` function should NOT load the complete intermediate indices or keep the whole merged index in memory.
- The `retrieve` function should order the query terms by postings list length.
- Both the `merge` function and the `sorted_intersect` function should NOT use the built in set operations  while merging postings lists or intersecting sorted lists.
- Timings for the `retrieve` function should be within a time range. This will be ensured if you do NOT iterate through your index instead of seeking just the query terms, as this will significantly increase the time complexity. 
- Your compression algorithm should indeed achieve a compressed index, i.e., you should see a significant index size decrease.

Let's import a few necessary packages

In [125]:
import sys
import pickle as pkl
import array
import os
import timeit
import contextlib

# Corpus

The corpus you will be working with for this assignment contains webpages from the cs.vt.edu, vt.edu, illinois.edu and cs.illinois.edu domains. The data for this assignment are included as a .zip file. There are 4 sub-directories under the data directory. We will also remove HTML tags so that each webpage is a sequence of space-delimited words. Each consecutive span of non-space characters consists of a word token in the corpus. 

In [126]:
import zipfile

data_dir = 'hw1data'
zip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')
zip_ref.extractall()
zip_ref.close()
sorted(os.listdir('hw1data'))

['0', '1', '2', '3', '4', '5', '6']

In [127]:
# Strip of HTML tags from all files and lowercase all words. This might take a while.
from bs4 import BeautifulSoup
from tqdm import tqdm #progress bars

new_data_dir = 'hw1dataraw'
os.makedirs(new_data_dir, exist_ok=True)

def removeTags(soup):
    for item in soup.find_all('span', 'sr-only'):
        item.decompose()
    return soup

for (dirpath, dirnames, filenames) in os.walk(data_dir):
    print(f'Going over {dirpath} ...')
    for filename in tqdm(filenames, total=len(filenames)):
        inF = open(os.sep.join([dirpath, filename]),'rb')
        soup = BeautifulSoup(inF, features="html.parser")
        soup = removeTags(soup)
        text = soup.get_text(strip=True, separator = " ").lower()
        newsubdir = dirpath.replace(data_dir, new_data_dir)
        os.makedirs(newsubdir, exist_ok=True)
        outF = open(os.sep.join([newsubdir, filename]), "w", encoding = 'utf-8')
        outF.write(text)
        outF.close()
        inF.close()

data_dir = new_data_dir # point data_dir to the new dir.
filename = 'hw1dataraw/6/apply.html'
soup = BeautifulSoup(open(filename,'r'), features="html.parser")
soup = removeTags(soup)
print(soup.get_text(strip=True, separator = " ").lower()) 

0it [00:00, ?it/s]
  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Going over hw1data ...
Going over hw1data\0 ...


  2%|█▋                                                                                | 2/100 [00:00<00:06, 14.23it/s]

Going over hw1data\1 ...


  2%|█▋                                                                                | 2/100 [00:00<00:06, 15.72it/s]

...

Going over hw1data\5 ...


  2%|█▋                                                                                | 2/100 [00:00<00:06, 15.36it/s]

Going over hw1data\6 ...


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.31it/s]

(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new date().gettime(),event:'gtm.js'});var f=d.getelementsbytagname(s)[0],
j=d.createelement(s),dl=l!='datalayer'?'&l='+l:'';j.async=true;j.src=
'//www.googletagmanager.com/gtm.js?id='+i+dl;f.parentnode.insertbefore(j,f);
})(window,document,'script','datalayer','gtm-3sqhdc'); (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
      new date().gettime(),event:'gtm.js'});var f=d.getelementsbytagname(s)[0],
      j=d.createelement(s),dl=l!='datalayer'?'&l='+l:'';j.async=true;j.src=
      'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentnode.insertbefore(j,f);
      })(window,document,'script','datalayer','gtm-p6f85q5'); apply | virginia tech skip to main content skip to search virginia techâ® universal access universal access options report a barrier accessibility portal pause all background videos underline all links apply visit give shop hokie gear apparel, clothing, gear and merchandise hokie shop university bo

We will create the directory for where the index will be stored (`index_dir`) and a separate folder `tmp` that will contain some temporary files. A small test corpus in subfolder `testdata` is included in the HW folder, which you can use for code testing purposes, and faster implementation cycles. The index for the `testdata` will be stored in `test_index_dir`

In [128]:
try: 
    os.mkdir('index_dir')
except FileExistsError:
    pass
try: 
    os.mkdir('tmp')
except FileExistsError:
    pass
try: 
    os.mkdir('test_index_dir')
except FileExistsError:
    pass

# Build an uncompressed index

The first part of this assignment is to build an inverted index of this corpus, and implement Boolean conjunctive queries. **In particular, you need to implement the blocked sort-based indexing (BSBI) algorithm described in [Section 4.2](http://nlp.stanford.edu/IR-book/pdf/04const.pdf) of the textbook.** To fully understand the details of BSBI, we recommend that you read Section 4.2 .

> Remember from lectures, to construct an index, first we assemble all term-docID pairs, where each document is represented with a unique serial numbers instead of strings. Then, we sort term-docID pairs by the term and docID. Finally, we organize docIDs for each term into a postings list and compute statistics like term and document frequency. To make the index more efficient, we can also represent terms as termIDs (unique serial numbers instead of strings). The dictionary that maps terms to termIDs can be built on the fly as we are processing the document collection. For small collections, the whole index construction can be done in memory. Here, we will implement large-scale methods that require use of secondary storage (disk).

## IdMap

Let us first build a helper class `IdMap`, which maps strings to numeric ids (and vice-versa). This will be useful for mapping terms to termIDs and docs to docIDs. Data structures used include: i) dictionary (from strings to numeric ids) and ii) lists (from numeric ids to strings).


In the following code, fill in the functions `_get_str` and `_get_id` . Notice that accessing items can be done by `__getitem__` which gets the correct mapping depending on the type of the key. We will later on incorporate functionality to add a string if it doesn't already exist in the map.   



In [129]:

class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        return self.id_to_str[i]
        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        if s not in self.str_to_id:
            self.str_to_id[s] = self.__len__()
            self.id_to_str.append(s)
            return self.str_to_id[s]
        else:
            return self.str_to_id[s]
        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

Make sure your code passes the following test cases:

In [130]:
testIdMap = IdMap()
assert testIdMap['a'] == 0, "Unable to add a new string to the IdMap"
assert testIdMap['bcd'] == 1, "Unable to add a new string to the IdMap"
assert testIdMap['a'] == 0, "Unable to retrieve the id of an existing string"
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

From now on you can use the `testdata` to write your own test cases, if you want to make sure that your code is working as expected. 

## Encoding Postings List as bytearrays

In order to write and read lists of postings (docIDs) efficiently from the disk, we store them as bytearrays. The provided`UncompressedPostings` class which contains static encode and decode functions. In the next task you'll be required to implement compressed versions with the same inferface. 

Some links for reference:
1. https://docs.python.org/3/library/array.html
2. https://pymotw.com/3/array/#module-array
3. https://realpython.com/instance-class-and-static-methods-demystified/

In [131]:
class UncompressedPostings:
    
    @staticmethod
    def encode(postings_list):
        """Encodes postings_list into a stream of bytes

        Parameters
        ----------
        postings_list: List[int] of docIDs (postings)
            
        Returns
        -------
        bytes: bytearray representing integers in the postings_list
        """
        return array.array('L', postings_list).tobytes()
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes postings_list from a stream of bytes
        
        Parameters
        ----------
        encoded_postings_list: bytearray representing encoded postings list as output by encode function
            
        Returns
        -------
        decoded_postings_list: Decoded List[int] of docIDs from encoded_postings_list
        """
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()

To illustrate how it works, run the following cell

In [132]:
x = UncompressedPostings.encode([1,2,3])
print(x)
print(UncompressedPostings.decode(x))

b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'
[1, 2, 3]


## Inverted Index on Disk

As mentioned in lectures, inverted indices utilize external sorting algorithms (that uses disk). For efficiency, any such sorting algorithm needs to minize the number of random disks seeks, i.e., sequential disk reads are much faster than random seeks.

In this section we provide a base class `InvertedIndex` which would be subsequently subclassed into `InvertedIndexWriter`, `InvertedIndexIterator` and `InvertedIndexMapper`.

In [133]:
class InvertedIndex:
    """Implements efficient reads and writes of an inverted index to disk
    
    Attributes
    ----------
    postings_dict: Dictionary mapping: termID->(start_position_in_index_file, 
                                                number_of_postings_in_list,
                                                length_in_bytes_of_postings_list)
        
        This is a dictionary that maps from termIDs to a 3-tuple of metadata that is helpful in reading 
        and writing the postings in the index file to/from disk. This mapping is supposed to be kept in memory. 
        
        start_position_in_index_file is the position (in bytes) of the postings list in the index file
        number_of_postings_in_list is the number of postings (docIDs) in the postings list
        length_in_bytes_of_postings_list is the length of the byte encoding of the postings list
    
    terms: List[int]
        A list of termIDs to remember the order in which terms and their postings lists were added to index. 
    """
    
    def __init__(self, index_name, postings_encoding=None, directory=''):
        """
        Parameters
        ----------
        index_name (str): Name used to store files related to the index 
        postings_encoding: A class implementing static methods for encoding and decoding lists of integers. 
                           Default is None, which gets replaced with UncompressedPostings
        directory (str): Directory where the index files will be stored
        """
        self.index_file_path = os.path.join(directory, index_name+'.index')
        self.metadata_file_path = os.path.join(directory, index_name+'.dict')
        if postings_encoding is None:
            self.postings_encoding = UncompressedPostings
        else:
            self.postings_encoding = postings_encoding
        self.directory = directory
        self.postings_dict = {}
        self.terms = []         

    def __enter__(self):
        """Opens the index_file and loads metadata upon entering the context"""
        # Open the index file
        self.index_file = open(self.index_file_path, 'rb+')
        # Load the postings dict and terms from the metadata file
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()                       
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        """Closes the index_file and saves metadata upon exiting the context"""
        # Close the index file
        self.index_file.close()
        # Write the postings dict and terms to the metadata file
        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

Note that `__enter__` and `__exit__` functions make `InvertedIndexWriter` a [context manager](https://docs.python.org/3/library/contextlib.html) that allows using the `with` statement just like native file I/O in python. 

```python
with InvertedIndexWriter('test', directory='tmp/') as index:
    # Some code here
```

Additional references on context managers in python (although thorough understanding is not neeed for this HW):
1. https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/
2. http://arnavk.com/posts/python-context-managers/

## Indexing

BSBI segments the collection into blocks of equal size, sorts the termID-docID pairs of each part in memory, and stores intermediate sorted results on disk. Finally, all intermediate results are merged as the final index. 

Here, we treat each sub-directory as a block, i.e.,  small enough to be stored in memory, and only load one block in memory at a time when we build the index. We will build upon the `BSBIIndex` class. The `index` function provides the skeleton for BSBI. 

The next HW task is to implement `parse_block`, `invert_write` and `merge` functions in the subsequent sections.

In [134]:
class BSBIIndex:
    """ 
    Attributes
    ----------
    term_id_map(IdMap): For mapping terms to termIDs
    doc_id_map(IdMap): For mapping relative paths of documents to docIDs
    data_dir(str): Path to data
    output_dir(str): Path to output index files
    index_name(str): Name assigned to index
    postings_encoding: Encoding used for storing the postings. The default (None) implies UncompressedPostings
    """
    def __init__(self, data_dir, output_dir, index_name = "BSBI", postings_encoding = None):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding
        # Stores names of intermediate indices
        self.intermediate_indices = []
        
    def save(self):
        """Dumps doc_id_map and term_id_map into output directory"""
        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pkl.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pkl.dump(self.doc_id_map, f)
    
    def load(self):
        """Loads doc_id_map and term_id_map from output directory"""
        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pkl.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pkl.load(f)
            
    def index(self):
        """
        This function loops through the data directories, calls parse_block to parse the documents, 
        calls invert_write, which inverts each block and writes to a new index and then saves the 
        id maps and calls merge on the intermediate indices.
        """
        for block_dir_relative in sorted(next(os.walk(self.data_dir))[1]):
            td_pairs = self.parse_block(block_dir_relative)
            index_id = 'index_'+block_dir_relative
            self.intermediate_indices.append(index_id)
            with InvertedIndexWriter(index_id, directory=self.output_dir, 
                                     postings_encoding=self.postings_encoding) as index:
                self.invert_write(td_pairs, index)
                td_pairs = None
        self.save()
        with InvertedIndexWriter(self.index_name, directory=self.output_dir, 
                                 postings_encoding=self.postings_encoding) as merged_index:
            with contextlib.ExitStack() as stack:
                indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=self.output_dir, 
                                          postings_encoding=self.postings_encoding)) 
                 for index_id in self.intermediate_indices]
                self.merge(indices, merged_index)

### Parsing

The function `parse_block` parses documents into termID-docID pairs and accumulates the pairs in memory until a block of a fixed size is full. Here, we treat each sub-directory as a block (`parse_block` expects the path to the sub-directory). You can assume that the full path of the files are unique. That means that while the individual file names are unique within each sub-directory, this may not necessarily be the case across sub-directories. 

*Disclaimer: it may seem that `BSBIIndex` inherits `BSBIIndex` (same class), but this is just a convenient way to split class definitions into two parts, such that we can add a new method to an existing class. This note is to avoid confusion on how we used the same name for two classes.*

In [147]:
class BSBIIndex(BSBIIndex):            
    def parse_block(self, block_dir_relative):
        """Parses a tokenized text file into termID-docID pairs
        
        Parameters
        ----------
        block_dir_relative : str Relative Path to the directory that contains the files for the block
        
        Returns
        -------
        parseLst: List[Tuple[Int, Int]] 
        Returns all the termID-docID pairs extracted from the block
        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        These persist across calls to parse_block
        """
        ### Begin your code
        parseList = []
        file_rel_path = os.path.join(self.data_dir, block_dir_relative)
        list_of_files = os.listdir(file_rel_path)
        for files in list_of_files:
#             index_of_file = self.doc_id_map[os.path.join(block_dir_relative, files)]
            index_of_file = self.doc_id_map[files]
            with open(os.path.join(file_rel_path, files), 'r', encoding="utf-8") as file:
                for line in file.readlines():
                    # line = line.translate(str.maketrans('', '', string.punctuation))
                    for word in line.lower().split():
                        term_id = self.term_id_map[word]
                        parseList.append((term_id, index_of_file))

#         self.term_id_map._dump()
        return parseList
        ### End your code

See if the function works as expected on the toy data.

In [140]:
with open('testdata/0/fine.txt', 'r') as f:
    print(f.read())
with open('testdata/0/hello.txt', 'r') as f:
    print(f.read())

i'm fine , thank you

hi hi
how are you ?



In [141]:
BSBI_instance = BSBIIndex(data_dir='testdata', output_dir = 'tmp/', index_name = 'test')
BSBI_instance.parse_block('0')

[[0, 0],
 [1, 0],
 [2, 0],
 [3, 0],
 [4, 0],
 [5, 1],
 [5, 1],
 [6, 1],
 [7, 1],
 [4, 1],
 [8, 1]]

Does `parse_block` work as expected on a block of the `testdata`? 
Write a test to make sure that a given word gets the same id each time it appears.

In [142]:
### Begin your code
assert BSBI_instance.parse_block('1') == BSBI_instance.parse_block('1')
### End your code

### Inversion

However, we first implement the class `InvertedIndexWriter`, which provides an append function, just like appending to a list, with the difference that the postings_list isn't stored in memory but is directly written to disk. Next, we will add a function `invert_write` that inverts a block and writes it to disk. Inversion involves first sorting the termID-docID pairs and then collect all of pairs with the same termID into a postings list, where a posting is simply a docID. The result is an inverted index for the block, which is finally written to disk. You might find it helpful to read the [Python I/O docs](https://docs.python.org/3/tutorial/inputoutput.html) for information about appending to the end of a file.

In [143]:
class InvertedIndexWriter(InvertedIndex):
    """"""
    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')              
        return self

    def append(self, term, postings_list):
        """Appends the term and postings_list to end of the index file.
        
        This function should:
        1. Encode the postings_list using self.postings_encoding
        2. Store metadata in the form of self.terms and self.postings_dict
           Note that self.postings_dict maps termID to a tuple of 
           (start_position_in_index_file, number_of_postings_in_list, length_in_bytes_of_postings_list)
        3. Appends the bytestream to the index file on disk
        
        Parameters
        ----------
        term: term or termID is the unique identifier for the term
        postings_list: List[Int] of docIDs where the term appears
        """
        ### Begin your code
        encoded_postings_list = self.postings_encoding.encode(postings_list)
        start_position_in_index_file = self.index_file.seek(0, 2)
        length_in_bytes_of_postings_list = self.index_file.write(encoded_postings_list)
        self.terms.append(term)
        self.postings_dict[term] = (start_position_in_index_file, len(postings_list), length_in_bytes_of_postings_list)
        ### End your code

Make sure you pass the following asserts before moving forward:

In [144]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect"
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"

Now we will implement `invert_write`, which takes as input the termID-docID pairs created with parse_block and writes them to the given index directory, by using `InvertedIndexWriter` to write to disk.

In [52]:
from collections import defaultdict

In [145]:
class BSBIIndex(BSBIIndex):
    def invert_write(self, td_pairs, index):
        """Inverts termID-docID pairs into postings_lists and writes them to the given index
        
        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]  (List of termID-docID pairs)
        index: InvertedIndexWriter  (Inverted index on disk corresponding to the block)   
        """
        ### Begin your code
        td_dict = defaultdict(list)
        for t, d in td_pairs:
            td_dict[t].append(d)
        for t in sorted(td_dict.keys()):
            p_list = sorted(td_dict[t])
            index.append(t, sorted(p_list))
        ### End your code

Write some tests as you did for the `InvertedIndexWriter`, to test this on a block of the `testdata` (using the `tmp` directory) and see what the inverted index contains. 

In [54]:
### Begin your code
BSBI_instance = BSBIIndex(data_dir='testdata', output_dir = 'tmp/', index_name = 'test')
with InvertedIndexWriter('test', directory='tmp/') as index:
    test_pair = [[2,3], [2,5], [1,2], [1,4], [1,3], [2,4]]
    BSBI_instance.invert_write(test_pair, index)
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect"
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"
### End your code

### Merging

Now we need to merge the ten blocks into one large merged index. We open all block files simultaneously, and maintain small read buffers for the ten blocks we are reading and a write buffer for the final merged index we are writing. We can iterate through the file while reading just one postings list at a time from the disk. We subclass `InvertedIndex` into `InvertedIndexIterator` to construct this iterator. 

In [55]:
class InvertedIndexIterator(InvertedIndex):
    """"""
    def __enter__(self):
        """Adds an initialization_hook to the __enter__ function of super class
        """
        super().__enter__()
        self._initialization_hook()
        return self

    def _initialization_hook(self):
        """Use this function to initialize the iterator
        """
        ### Begin your code
        self.term_position = 0
        ### End your code

    def __iter__(self): 
        return self
    
    def __next__(self):
        """Returns the next (term, postings_list) pair in the index.
        Note: This function should only read a small amount of data from the index file. 
        In particular, you should not try to maintain the full index file in memory.
        """
        ### Begin your code
        if self.term_position < len(self.terms):
            term = self.terms[self.term_position]
            self.term_position += 1
            start_index, _, length_of_byte_array = self.postings_dict[term]
            self.index_file.seek(start_index)
            postings = self.postings_encoding.decode(self.index_file.read(length_of_byte_array))
            return term, postings
        else:
            raise StopIteration
        ### End your code

    def delete_from_disk(self):
        """Marks the index for deletion upon exit. Useful for temporary indices
        """
        self.delete_upon_exit = True

    def __exit__(self, exception_type, exception_value, traceback):
        """Delete the index file upon exiting the context along with the 
        functions of the super class __exit__ function"""
        self.index_file.close()
        if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
            os.remove(self.index_file_path)
            os.remove(self.metadata_file_path)
        else:
            with open(self.metadata_file_path, 'wb') as f:
                pkl.dump([self.postings_dict, self.terms], f)

Let's test this. Write a test that constructs a small inverted index, writes them to disk (in `tmp` folder) with an `InvertedIndexWriter`, then uses an `InvertedIndexIterator` to iterate over the inverted index.

In [56]:
### Begin your code
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
with InvertedIndexIterator('test', directory='tmp/') as index:
    assert next(index) == (1, [2, 3, 4])
    assert next(index) == (2, [3, 4, 5])
### End your code

During merging, in each iteration, we select the lowest termID that has not been processed yet (using a priority queue or a similar data structure). All postings lists for this termID are read and merged, and the merged list is written back to disk. Each read buffer is refilled from its file when necessary. We'll use the `InvertedIndexIterator` to do the reading part and `InvertedIndexWriter` to write the merged postings list. 

Your task is to write the logic for merging *opened* `InvertedIndexIterator` objects and writing one postings list at a time into into a single `InvertedIndexWriter` object. 

Since we know that the postings are sorted, we can appropriately merge them in sorted order in linear time. In fact `heapq` ([documentation](https://docs.python.org/3.0/library/heapq.html)) is a standard python module that provides an implementation of the heap queue algorithm. In particular it contains a `heapq.merge` utility function which merges multiple sorted inputs into a single sorted output and returns an iterator over the sorted values. Not only can this be handy with merging postings lists, but also with merging the inverted indices. To get you started on using the `heapq.merge` function, we've provided a sample usage of the function. The two lists contain animals/birds sorted by their average life span. We want to merge the two lists.

In [57]:
import heapq
animal_lifespans = [('Giraffe', 28), ('Rhinoceros', 40), ('Indian Elephant', 70), ('Golden Eagle', 80)]
tree_lifespans = [('Gray Birch', 50), ('Black Willow', 70), ('Basswood', 100), ('Bald Cypress', 600)]
lifespan_lists = [animal_lifespans, tree_lifespans]
for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
    print(merged_item)

('Giraffe', 28)
('Rhinoceros', 40)
('Gray Birch', 50)
('Indian Elephant', 70)
('Black Willow', 70)
('Golden Eagle', 80)
('Basswood', 100)
('Bald Cypress', 600)


Notice the use of `*` to unpack `lifespan_lists` as arguments and the `lambda` function to represent the key based on which sorting is performed. 

We list here some references and tutorials:
1. Unpacking lists [python documentation](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists) and a [tutorial](https://www.geeksforgeeks.org/packing-and-unpacking-arguments-in-python/)
2. Lambda expressions [python documentation](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) and a [tutorial](https://www.afternerd.com/blog/python-lambdas/)

Now complete the `merge` function below:

In [58]:
import heapq
class BSBIIndex(BSBIIndex):
    def merge(self, indices, merged_index):
        """Merges multiple inverted indices into a single index
        
        Parameters
        ----------
        indices: List[InvertedIndexIterator]
            A list of InvertedIndexIterator objects, each representing an iterable inverted index for a block
        merged_index: InvertedIndexWriter
            An instance of InvertedIndexWriter into which each merged postings list is written out one at a time
        """
        ### Begin your code
        last_term = last_posting = None
        for curr_term, curr_postings in heapq.merge(*indices):
            if curr_term != last_term:
                if last_term:
                    last_posting = list(sorted(set(last_posting)))
                    merged_index.append(last_term, last_posting)
                last_term = curr_term
                last_posting = curr_postings
            else:
                last_posting += curr_postings
        if last_term:
            last_posting = list(sorted(set(last_posting)))
            merged_index.append(last_term, last_posting)
        ### End your code

Make sure it works without errors on `testdata` (using `test_index_dir` as output directory)

In [59]:
BSBI_instance = BSBIIndex(data_dir='testdata', output_dir = 'test_index_dir', )
BSBI_instance.index()

Now lets index the full corpus (using `index_dir` as output directory)

In [60]:
BSBI_instance = BSBIIndex(data_dir=data_dir, output_dir = 'index_dir', )
BSBI_instance.index()

If you face any issues or errors with the merging part, use the following code to debug it. 

In [ ]:
BSBI_instance = BSBIIndex(data_dir=data_dir, output_dir = 'index_dir', )
BSBI_instance.intermediate_indices = [f'index_{i}' for i in os.listdir(data_dir)]
with InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:
    with contextlib.ExitStack() as stack:
        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]
        BSBI_instance.merge(indices, merged_index)

## Boolean conjunctive retrieval

Your task here is to implement a function `retrieve` to BSBIIndex, which given a query string consisting of space-delimited tokens, returns a list of documents that contain each of the tokens in the query. However, note that we do not want to be iterating through the index, nor loading the entire index to find the relevant terms. First we'll implement `InvertedIndexMapper` which subclasses `InvertedIndex` to add functionality for retrieving postings corresponding to a particular term by seeking to that location in the file.

In [62]:
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)
    
    def _get_postings_list(self, term):
        """Gets a postings list (of docIds) for `term`. This function SHOULD NOT iterate through the index file, 
        but instead read only the bytes from the index file corresponding to the postings list for the requested term.
        """
        ### Begin your code
        start_index, _, length_of_byte_array = self.postings_dict[term]
        self.index_file.seek(start_index)
        postings = self.postings_encoding.decode(self.index_file.read(length_of_byte_array))
        return postings
        ### End your code

Write a few tests to check your `_get_postings_list` implementation.

In [63]:
### Begin your code
with InvertedIndexMapper('test', directory='tmp/') as mapper:
    assert mapper[1] == [2,3,4]
### End your code

We can now obtain postings lists corresponding to the query terms, but to create a boolean query, we need to intersect them. We can use the fact that these lists are pre-sorted to intersect them efficiently. 

Your next task is to implement the `sorted_intersect` function that takes two sorted lists and returns a sorted intersection of the elements.  

In [64]:
def sorted_intersect(list1, list2):
    """Intersects two (ascending) sorted lists and returns the sorted result
    
    Parameters
    ----------
    list1: List[Comparable]
    list2: List[Comparable]
    Sorted lists to be intersected
        
    Returns
    -------
    List[Comparable] Sorted intersection        
    """
    ### Begin your code
    return list(sorted(set(list1).intersection(set(list2))))
    ### End your code

You can also write a few test cases to check that it works properly

In [65]:
### Begin your code
l1 = [0,1,2,3,4,5,6]
l2 = [4,5,6]
print(sorted_intersect(l1, l2))
### End your code

[4, 5, 6]


Here, the task is to write the `retrieve` function using `InvertedIndexMapper` and `sorted_intersect`.
Note that `retrieve` should NOT throw errors for terms not in corpus. You can take a look at exception handling in python [here](https://docs.python.org/3/tutorial/errors.html).

In [123]:
class BSBIIndex(BSBIIndex):
    def retrieve(self, query):
        """Retrieves the documents corresponding to the conjunctive query
        
        Parameters
        ----------
        query: str  Space separated list of query tokens
            
        Result
        ------
        List[str] Sorted list of documents which contains each of the query tokens. 
        Should be empty if no documents are found. Should NOT throw errors for terms not in corpus
        """
        if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
            self.load()

        ### Begin your code
        """ans = None
        with InvertedIndexMapper(self.index_name, directory=self.output_dir, postings_encoding= self.postings_encoding) as mapper:
            for term in query.split():
                try:
                    termid = self.term_id_map[term]
                    postings_list = mapper[termid]
                    if not ans:
                        ans = postings_list
                    else:
                        ans = sorted_intersect(ans, postings_list)
                except KeyError:
                    return []
        ans = [self.doc_id_map[docid] for docid in ans]
        return ans"""
    
        with InvertedIndexMapper(self.index_name, directory=self.output_dir, 
                                 postings_encoding=
                                 self.postings_encoding) as mapper:
            result = None
            for term in query.split():
                term_id = self.term_id_map.str_to_id.get(term)
                if not term_id:
                    return []
                r = mapper[term_id]
                if result is None:
                    result = r
                else:
                    result = sorted_intersect(result, r)
        return [self.doc_id_map[r] for r in result]
        ### End your code

Let us test if our index works on the real corpus with a simple query

In [124]:
BSBI_instance = BSBIIndex(data_dir=data_dir, output_dir = 'index_dir', )
BSBI_instance.retrieve('career fair')

ModuleNotFoundError: No module named 'HW1'

We can also check if indeed one of the retrieved pages contains the query terms, by reading a file as follows:

In [70]:
with open("hw1dataraw/3/fall-career-fair.html", 'r') as f:
    print(f.read())

(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
      new date().gettime(),event:'gtm.js'});var f=d.getelementsbytagname(s)[0],
      j=d.createelement(s),dl=l!='datalayer'?'&l='+l:'';j.async=true;j.src=
      'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentnode.insertbefore(j,f);
      })(window,document,'script','datalayer','gtm-p6f85q5'); cs|source offers in-person and virtual connections with companies   | computer science | virginia tech skip to main content skip to search virginia techÂ® universal access universal access options report a barrier accessibility portal pause all background videos underline all links apply visit give shop hokie gear apparel, clothing, gear and merchandise hokie shop university bookstore, merchandise and gifts hokie license plates part of every virginia tech plate purchase funds scholarships resources for future students current students parents and families faculty and staff alumni industry and partners computer science menu col

# Building a compressed index

In this section, we will build a compressed index using gap encoding with variable byte encoding for each gap.

First, let's take a look at some useful python math operations: 

In [71]:
# Remainder (modulo) operator %
print("10 % 2 = ", 10 % 2)
print("10 % 3 = ", 10 % 3)

# Integer division in Python 3 is done by using two slash signs
print("10 / 3 = ", 10 / 3)
print("10 // 3 = ", 10 // 3)

10 % 2 =  0
10 % 3 =  1
10 / 3 =  3.3333333333333335
10 // 3 =  3


Your taks is to fill in the following `CompressedPostings` class which we'll use as a drop-in replacement for `UncompressedPostings`. To understand in detail gap encoding (with variable byte encoding for each gap), we suggest that you revisit the corresponding lecture video and [Chapter 5](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf) of the textbook. 

In [75]:
824//128

6

In [79]:
class CompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code
    @staticmethod
    def VBEncodeNumber(number):
        byte = []
        while True:
            byte.append(number % 128)
            if number < 128:
                break
            number //= 128
        byte[0] += 128
        return byte[::-1]

    @staticmethod
    def VBEncode(postings_list):
        bytestream = []
        for n in postings_list:
            bytestream.extend(CompressedPostings.VBEncodeNumber(n))
        return bytestream

    @staticmethod
    def VBDecode(bytestream):
        postings_list = []
        n = 0
        for b in bytestream:
            if b < 128:
                n = 128*n + b
            else:
                n = 128*n + b - 128
                postings_list.append(n)
                n = 0
        return postings_list
    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` using gap encoding with variable byte 
        encoding for each gap
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
            (as produced by `array.tobytes` function)
        """
        ### Begin your code
        p = postings_list.copy()
        for i in range(1, len(p))[::-1]:
            p[i] -= p[i-1]
        vb = CompressedPostings.VBEncode(p)
        return array.array('B', vb).tobytes()
        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docIds)
        """
        ### Begin your code
        vb = array.array('B')
        vb.frombytes(encoded_postings_list)
        postings_list = CompressedPostings.VBDecode(vb.tolist())
        for i in range(1, len(postings_list)):
            postings_list[i] += postings_list[i-1]
        return postings_list

You can write test cases for any helper methods implemented

In [90]:
### Begin your code
assert CompressedPostings.VBEncodeNumber(786) == [6, 146]
assert CompressedPostings.VBEncode([36]) == [164]
assert CompressedPostings.VBDecode([164]) == [36]
### End your code

We have provided a helper function for testing whether an encoded postings list is decoded correctly.

In [91]:
def test_encode_decode(l):
    e = CompressedPostings.encode(l)
    d = CompressedPostings.decode(e)
    assert d == l
    print(l, e)

Write a few test cases to make sure the postings list compression and decompression is being done correctly.

In [94]:
### Begin your code
test_encode_decode([800, 900, 10000])
### End your code

[800, 900, 10000] b'\x06\xa0\xe4G\x8c'


Now let's create a new folder to store the compressed index

In [95]:
try: 
    os.mkdir('index_dir_compressed')
except FileExistsError:
    pass

In [96]:
BSBI_instance_compressed = BSBIIndex(data_dir=data_dir, output_dir='index_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.index()

In [97]:
BSBI_instance_compressed.retrieve('career fair')

['celebration',
 'department-college-and-campus-events',
 'dskatz',
 'featured-lectures',
 'nskim',
 'rutameht',
 'values',
 'accessibility.html',
 'honor-code',
 'index.html',
 'jugal',
 'theory-and-algorithms',
 'cs-source-events.html',
 'fall-career-fair.html',
 'featured-spotlight.html',
 'property-management.html',
 'site-map.html',
 'graduate.html',
 'construction-engineering-and-management.html']

And we are done with our search engine implementation!


## Running time analysis (Optional, NOT graded)

In this section you can write queries to evaluate the running time characteristics of `retrieve` and use it to for in-practice understanding of the key optimizations decisions when designing a retrieval algorithm. Note that, due to the very small data collection size, timings may not be perfect (and sometimes results may vary). However, you should still be able to see some timing differences often. Also, note that the following code cells assume there exists an indexed BSBIIndex object.

Python offers a convenient timing module `%timeit` ([documentation](https://docs.python.org/3/library/timeit.html)).

In [98]:
%timeit BSBI_instance.retrieve('virginia tech')

50.7 ms ± 6.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Can you **replace or add** a term in the query so that it runs slower? Time your query to see if indeed is slower.

In [105]:
%timeit BSBI_instance.retrieve('virginia tech')
### Begin your code
%timeit BSBI_instance.retrieve('the')
### End your code

53.4 ms ± 4.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
61.7 ms ± 17.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Why did the retrieve function get slower?**
> *Since we are not removing stopwords (words that appear in high frequency but do not necessarily contribute much for retrieval) the merge operation takes a lot of time to create an index for them since their frequency is higher. Hence it takes more time for those terms.*

Construct a new query by adding a term to the previous query that makes it go faster.

In [ ]:
### Begin your code 

### End your code

**Where does the increase in speed come from?** 

> *Type your answer here*

## Index size

In this section we will look at the size of index on disk.

### Uncompressed index

Let us look at the size of the merged index (`.index`) file 

In [101]:
print("Size of index", os.path.getsize("index_dir/BSBI.index"), 'bytes')

Size of index 1648172 bytes


The code above queries the file system to compute the size of the index. 

Show that you can compute the same answer by looking at the `postings_dict` object directly. Your code should compute the size of the index file (in bytes) using the postings dict. You should use the word size (in bytes) and size information stored in the `postings_dict`. The resulting number should match the byte number above.

In [103]:
with open('index_dir/BSBI.dict', 'rb') as f:
    postings_dict, terms = pkl.load(f)
    ### Begin your code
    res = 0
    for v in postings_dict.values():
        res += v[-1]
    print(res)
    ### End your code

1648172


**Q1: Describe how you have computed the expected size based on the information stored in the postings dictionary.**

> *Since we save the metadata of the index in the postings_dictionary where we store a tuple of (start_index, number_of_postings, length_in_byte_array(using Compressed / Uncompressed Encodings)) we can straightaway add up all the lenghts of byte arrays to get the actual length of the index file.*

### Compressed Index

Let us look at the size of the compressed index (`.index`) file 

In [104]:
print("Size of compressed index", os.path.getsize("index_dir_compressed/BSBI.index"), 'bytes')

Size of compressed index 450840 bytes


**Q2: How would the size of the compressed index change if variable byte encoding was used on the docIDs instead of the gaps?**

> *If Variable Byte Encoding was used only on docIDs instead of gaps the size of the index will be larger. *

## Improving performance

Let's take a deeper look into improving performance. Think about your answers to the following questions. Note that some questions are subjective and may have multiple correct answers. 

**In this HW you useD each sub-directory as a block and build an index for one block at a time. Can you discuss the trade-off of different sizes of blocks? Is there a better way to work with limited memory when we want to minimize indexing time?**

> The block size to index is restricted by the memory capacity of the machine. If we use larger blocks then obiously the documents will be processed much faster and since we reduce the final merge operations. With a limited memory the best way to ensure minimizing index time is using blocks of size which fit and use up all the available memory efficiently.

**Q3: Is there a part of your indexing program that limits its scalability to larger datasets?**

> The term dictionary will always be in memory; as a result the system become handicapped for larger datasets if it is constrained by memory issues. 

**Q4: Describe other parts of the indexing process that you could optimize for indexing scalability and faster retrieval time.**

> We can preprocess data in such a way that more frequently appearing terms (stopwords) are preprocessed in such a way as they are assigned lesser priority since they can increase the time required for the merge operation while indexing. Hence data normalization can be a way in which this system can be optimized

In [118]:
#import nbimporter
#import os
#from HW1 import BSBIIndex, CompressedPostings

print("Testing BSBIIndex.")
BSBI_instance = BSBIIndex(data_dir='hw1dataraw', output_dir = 'index_dir', )
print("Building BSBIIndex ...")
BSBI_instance.index()
print("Done. Begin test cases ...")
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance.retrieve(query)]
        #print(my_results)
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip().split('/')[1]) for x in o.readlines()]
            print(set(my_results)-set(reference_results))
#             assert not set(my_results)-set(reference_results), "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())
print("BSBIIndex test done.") 


print("Testing Compressed BSBIIndex.")
BSBI_instance_compressed = BSBIIndex(data_dir='hw1dataraw', output_dir='index_dir_compressed', postings_encoding=CompressedPostings)
print("Building BSBIIndex compressed...")
BSBI_instance_compressed.index()
print("Done. Begin test cases ...")
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_compressed.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            print(set(my_results)-set(reference_results))
#             assert not set(my_results)-set(reference_results), "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())
print("Compressed BSBIIndex test done.") 

Testing BSBIIndex.
Building BSBIIndex ...
Done. Begin test cases ...
{'rankings.html', 'research.html', 'accessibility.html', 'academics.html', 'contact.html'}
Results match for query: virginia tech class
set()
Results match for query: many students
{'environmental-horticulture.html'}
Results match for query: the the
set()
Results match for query: illinois computer science
set()
Results match for query: admission vt
set()
Results match for query: virginia tech phd cs
set()
Results match for query: the department of computer science
set()
Results match for query: admission vt college
BSBIIndex test done.
Testing Compressed BSBIIndex.
Building BSBIIndex compressed...
Done. Begin test cases ...
{'vt-icorps.html', 'international-studies.html', 'current-students.html', 'Prospective.html', 'xj-meng.html', 'advertising.html', 'Undergradhandbook.html', 'job-opportunities.html', 'launch-of-new-hokie-owned-business-directory.html', 'Computer-Science-Press-Releases.html', 'accessibility.html', 'p

Results match for query: illinois computer science
{'campus-highlights.html', 'tribal-initatives.html', 'Prospective.html', 'undergraduate.html', 'ProspectiveStudents.html', 'Undergradhandbook.html', 'BSMS.html', 'site-map.html', 'virtual-vt-visit.html', 'FAQs.html'}
Results match for query: admission vt
{'Scholarships.html', 'CS4634.html', 'FAQs.html', 'shj', 'CS3704.html', 'CS2104.html', 'CS3714.html', 'CS3744.html', 'CS1124.html', 'NewStudentFAQ.html', 'CS4124.html', 'CS4414.html', 'StudentServices.html', 'fall-career-fair.html', 'CS4304.html', 'CS4654.html', 'CS1014.html', 'Degrees.html', 'COVID19.html', 'sp22grforceadd.html', 'CS4214.html', 'CS4884.html', 'CS4944.html', 'CS2304.html', 'CS4824.html', 'featured-lectures', 'BSMS.html', 'CS4624.html', 'Forms.html', 'CS3414.html', 'CS4264.html', 'CS3824.html', 'CS4284.html', 'OptionsCertificates.html', 'ContactsRoles.html', 'CS1044.html', 'Giving.html', 'CS3604.html', 'CS4254.html', 'Administrivia.html', 'CS1604.html', 'CS4644.html', '

In [113]:
for filename in os.listdir('dev_output'):
    with open(os.path.join('dev_output', filename), 'rb') as f:
        print(f.read())

b'\x00\x00\x00\x01Bud1\x00\x00\x10\x00\x00\x00\x08\x00\x00\x00\x10\x00\x00\x00\x00%\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\